# Analysis of non-naive datetime handling in Python
Let's say we live in Belgium.

Standard time is CET and we do DST so we move to CEST from 2023-03-26 until 2023-10-29.  
When we go to CEST that day only has 23 hours. That day, there is no 2 AM.  
When we go to CET that day then has 25 hours. That day, there is 2 AM and 2AM*.  
  
Imagine that we have a series of strings representing datetimes matching this behaviour:

In [5]:
str_datetimes = ['2023-01-01 00:00', '2023-01-01 01:00', '2023-01-01 02:00', '2023-01-01 03:00',
                '2023-03-25 21:00', '2023-03-25 22:00', '2023-03-25 23:00',
                '2023-03-26 00:00', '2023-03-26 01:00', '2023-03-26 03:00',
                '2023-10-28 21:00', '2023-10-28 22:00', '2023-10-28 23:00',
                '2023-10-29 00:00', '2023-10-29 01:00', '2023-10-29 02:00', '2023-10-29 02:00*', '2023-10-29 03:00',
                '2023-12-31 20:00', '2023-12-31 21:00', '2023-12-31 22:00', '2023-12-31 23:00']


Now let's see what happens if we parse these dates to datetime objects and convert them to UTC without any additional guidance:

In [6]:
from datetime import datetime
parsed_datetimes = [datetime.strptime(x, '%Y-%m-%d %H:%M') for x in str_datetimes]

ValueError: unconverted data remains: *

Indeed we get a value error due to the 2:00*.
We can solve this by omitting that behaviour.


In [7]:
parsed_datetimes = [datetime.strptime(x, '%Y-%m-%d %H:%M') for x in str_datetimes if '*' not in x]
parsed_datetimes

[datetime.datetime(2023, 1, 1, 0, 0),
 datetime.datetime(2023, 1, 1, 1, 0),
 datetime.datetime(2023, 1, 1, 2, 0),
 datetime.datetime(2023, 1, 1, 3, 0),
 datetime.datetime(2023, 3, 25, 21, 0),
 datetime.datetime(2023, 3, 25, 22, 0),
 datetime.datetime(2023, 3, 25, 23, 0),
 datetime.datetime(2023, 3, 26, 0, 0),
 datetime.datetime(2023, 3, 26, 1, 0),
 datetime.datetime(2023, 3, 26, 3, 0),
 datetime.datetime(2023, 10, 28, 21, 0),
 datetime.datetime(2023, 10, 28, 22, 0),
 datetime.datetime(2023, 10, 28, 23, 0),
 datetime.datetime(2023, 10, 29, 0, 0),
 datetime.datetime(2023, 10, 29, 1, 0),
 datetime.datetime(2023, 10, 29, 2, 0),
 datetime.datetime(2023, 10, 29, 3, 0),
 datetime.datetime(2023, 10, 29, 4, 0),
 datetime.datetime(2023, 12, 31, 20, 0),
 datetime.datetime(2023, 12, 31, 21, 0),
 datetime.datetime(2023, 12, 31, 22, 0),
 datetime.datetime(2023, 12, 31, 23, 0)]

When we use pandas we see that the same ValueError occurs:

In [13]:
import pandas as pd
df = pd.DataFrame({'dt_str': str_datetimes})
df['dt_dt'] = pd.to_datetime(df['dt_str'], format='%Y-%m-%d %H:%M')



ValueError: unconverted data remains when parsing with format "%Y-%m-%d %H:%M": "*", at position 16. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

But pandas lets us coerce errors. This introduces NA into the dataframe.

In [15]:
df['dt_dt'] = pd.to_datetime(df['dt_str'], format='%Y-%m-%d %H:%M', errors='coerce')
df

,dt_str,dt_dt
0,2023-01-01 00:00,2023-01-01 00:00:00
1,2023-01-01 01:00,2023-01-01 01:00:00
2,2023-01-01 02:00,2023-01-01 02:00:00
3,2023-01-01 03:00,2023-01-01 03:00:00
4,2023-03-25 21:00,2023-03-25 21:00:00
5,2023-03-25 22:00,2023-03-25 22:00:00
6,2023-03-25 23:00,2023-03-25 23:00:00
7,2023-03-26 00:00,2023-03-26 00:00:00
8,2023-03-26 01:00,2023-03-26 01:00:00
9,2023-03-26 03:00,2023-03-26 03:00:00


We can now make the naive timezone object smarter by adding the timezone.  
Learnings
- the correct +1/+2 hour offset is applied for both CET/CEST
- it sees 2AM and on 29/10 and realises this is 'ambiguous' so it introduces another 'NaT' as per our setting

In [20]:
import pytz
df['dt_localized'] = df['dt_dt'].dt.tz_localize('Europe/Brussels', ambiguous='NaT')
df

,dt_str,dt_dt,dt_localized
0,2023-01-01 00:00,2023-01-01 00:00:00,2023-01-01 00:00:00+01:00
1,2023-01-01 01:00,2023-01-01 01:00:00,2023-01-01 01:00:00+01:00
2,2023-01-01 02:00,2023-01-01 02:00:00,2023-01-01 02:00:00+01:00
3,2023-01-01 03:00,2023-01-01 03:00:00,2023-01-01 03:00:00+01:00
4,2023-03-25 21:00,2023-03-25 21:00:00,2023-03-25 21:00:00+01:00
5,2023-03-25 22:00,2023-03-25 22:00:00,2023-03-25 22:00:00+01:00
6,2023-03-25 23:00,2023-03-25 23:00:00,2023-03-25 23:00:00+01:00
7,2023-03-26 00:00,2023-03-26 00:00:00,2023-03-26 00:00:00+01:00
8,2023-03-26 01:00,2023-03-26 01:00:00,2023-03-26 01:00:00+01:00
9,2023-03-26 03:00,2023-03-26 03:00:00,2023-03-26 03:00:00+02:00


We could maybe do this better by having 2 times 2AM and then having datetime 'infer' which one is CEST and which one is CET.
This gives GREAT results!!!

In [22]:
df['dt_dt2'] = pd.to_datetime(df['dt_str'].str.replace('*', ''), format='%Y-%m-%d %H:%M') # notice: no more errors!
df['dt_localized2'] = df['dt_dt2'].dt.tz_localize('Europe/Brussels', ambiguous='infer')
df

,dt_str,dt_dt,dt_localized,dt_dt2,dt_localized2
0,2023-01-01 00:00,2023-01-01 00:00:00,2023-01-01 00:00:00+01:00,2023-01-01 00:00:00,2023-01-01 00:00:00+01:00
1,2023-01-01 01:00,2023-01-01 01:00:00,2023-01-01 01:00:00+01:00,2023-01-01 01:00:00,2023-01-01 01:00:00+01:00
2,2023-01-01 02:00,2023-01-01 02:00:00,2023-01-01 02:00:00+01:00,2023-01-01 02:00:00,2023-01-01 02:00:00+01:00
3,2023-01-01 03:00,2023-01-01 03:00:00,2023-01-01 03:00:00+01:00,2023-01-01 03:00:00,2023-01-01 03:00:00+01:00
4,2023-03-25 21:00,2023-03-25 21:00:00,2023-03-25 21:00:00+01:00,2023-03-25 21:00:00,2023-03-25 21:00:00+01:00
5,2023-03-25 22:00,2023-03-25 22:00:00,2023-03-25 22:00:00+01:00,2023-03-25 22:00:00,2023-03-25 22:00:00+01:00
6,2023-03-25 23:00,2023-03-25 23:00:00,2023-03-25 23:00:00+01:00,2023-03-25 23:00:00,2023-03-25 23:00:00+01:00
7,2023-03-26 00:00,2023-03-26 00:00:00,2023-03-26 00:00:00+01:00,2023-03-26 00:00:00,2023-03-26 00:00:00+01:00
8,2023-03-26 01:00,2023-03-26 01:00:00,2023-03-26 01:00:00+01:00,2023-03-26 01:00:00,2023-03-26 01:00:00+01:00
9,2023-03-26 03:00,2023-03-26 03:00:00,2023-03-26 03:00:00+02:00,2023-03-26 03:00:00,2023-03-26 03:00:00+02:00


Now we can convert everything to UTC:

In [23]:
df['dt_utc'] = df['dt_localized2'].dt.tz_convert(pytz.utc)
df

,dt_str,dt_dt,dt_localized,dt_dt2,dt_localized2,dt_utc
0,2023-01-01 00:00,2023-01-01 00:00:00,2023-01-01 00:00:00+01:00,2023-01-01 00:00:00,2023-01-01 00:00:00+01:00,2022-12-31 23:00:00+00:00
1,2023-01-01 01:00,2023-01-01 01:00:00,2023-01-01 01:00:00+01:00,2023-01-01 01:00:00,2023-01-01 01:00:00+01:00,2023-01-01 00:00:00+00:00
2,2023-01-01 02:00,2023-01-01 02:00:00,2023-01-01 02:00:00+01:00,2023-01-01 02:00:00,2023-01-01 02:00:00+01:00,2023-01-01 01:00:00+00:00
3,2023-01-01 03:00,2023-01-01 03:00:00,2023-01-01 03:00:00+01:00,2023-01-01 03:00:00,2023-01-01 03:00:00+01:00,2023-01-01 02:00:00+00:00
4,2023-03-25 21:00,2023-03-25 21:00:00,2023-03-25 21:00:00+01:00,2023-03-25 21:00:00,2023-03-25 21:00:00+01:00,2023-03-25 20:00:00+00:00
5,2023-03-25 22:00,2023-03-25 22:00:00,2023-03-25 22:00:00+01:00,2023-03-25 22:00:00,2023-03-25 22:00:00+01:00,2023-03-25 21:00:00+00:00
6,2023-03-25 23:00,2023-03-25 23:00:00,2023-03-25 23:00:00+01:00,2023-03-25 23:00:00,2023-03-25 23:00:00+01:00,2023-03-25 22:00:00+00:00
7,2023-03-26 00:00,2023-03-26 00:00:00,2023-03-26 00:00:00+01:00,2023-03-26 00:00:00,2023-03-26 00:00:00+01:00,2023-03-25 23:00:00+00:00
8,2023-03-26 01:00,2023-03-26 01:00:00,2023-03-26 01:00:00+01:00,2023-03-26 01:00:00,2023-03-26 01:00:00+01:00,2023-03-26 00:00:00+00:00
9,2023-03-26 03:00,2023-03-26 03:00:00,2023-03-26 03:00:00+02:00,2023-03-26 03:00:00,2023-03-26 03:00:00+02:00,2023-03-26 01:00:00+00:00


Finally, let's focus on UTC, add some fake data...

In [30]:
dff = df[['dt_utc']]
dff['data'] = range(0,23)
dff

C:\Users\jorrit\AppData\Local\Temp\ipykernel_22140\2373239136.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['data'] = range(0,23)


,dt_utc,data
0,2022-12-31 23:00:00+00:00,0
1,2023-01-01 00:00:00+00:00,1
2,2023-01-01 01:00:00+00:00,2
3,2023-01-01 02:00:00+00:00,3
4,2023-03-25 20:00:00+00:00,4
5,2023-03-25 21:00:00+00:00,5
6,2023-03-25 22:00:00+00:00,6
7,2023-03-25 23:00:00+00:00,7
8,2023-03-26 00:00:00+00:00,8
9,2023-03-26 01:00:00+00:00,9


Then resample and interpolate

In [31]:
dff.set_index('dt_utc', inplace=True)
dfh = dff.resample('1H').interpolate()
dfh.reset_index(inplace=True)
dfh

,dt_utc,data
0,2022-12-31 23:00:00+00:00,0.000000
1,2023-01-01 00:00:00+00:00,1.000000
2,2023-01-01 01:00:00+00:00,2.000000
3,2023-01-01 02:00:00+00:00,3.000000
4,2023-01-01 03:00:00+00:00,3.000498
...,...,...
8755,2023-12-31 18:00:00+00:00,18.999346
8756,2023-12-31 19:00:00+00:00,19.000000
8757,2023-12-31 20:00:00+00:00,20.000000
8758,2023-12-31 21:00:00+00:00,21.000000
